<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/00_ExtraerCaracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
================================================================================
EXTRACTOR DE CARACTERÍSTICAS DE IMÁGENES
================================================================================

Este notebook extrae características completas de imágenes fotográficas
utilizando múltiples librerías especializadas. El análisis es independiente
del modelo de segmentación y genera un JSON con toda la información relevante.

CARACTERÍSTICAS ANALIZADAS:
- Metadatos EXIF (cámara, exposición, GPS, etc.)
- Estadísticas de color (RGB, HSV, LAB)
- Histogramas y distribuciones
- Texturas (Haralick features)
- Características visuales (bordes, esquinas, gradientes)
- Análisis de nitidez y calidad
- Dominancia de colores
- Información técnica de archivo

LIBRERÍAS UTILIZADAS:
- PIL/Pillow: Carga y metadatos básicos
- piexif: Metadatos EXIF completos
- OpenCV: Procesamiento de imagen
- scikit-image: Análisis avanzado
- mahotas: Texturas Haralick
- numpy/scipy: Cálculos matemáticos
- colorthief: Paleta de colores dominantes
- imagehash: Hashes perceptuales

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña
Fecha: 2025
================================================================================
"""

'\n================================================================================\nEXTRACTOR DE CARACTERÍSTICAS DE IMÁGENES\n================================================================================\n\nEste notebook extrae características completas de imágenes fotográficas \nutilizando múltiples librerías especializadas. El análisis es independiente \ndel modelo de segmentación y genera un JSON con toda la información relevante.\n\nCARACTERÍSTICAS ANALIZADAS:\n- Metadatos EXIF (cámara, exposición, GPS, etc.)\n- Estadísticas de color (RGB, HSV, LAB)\n- Histogramas y distribuciones\n- Texturas (Haralick features)\n- Características visuales (bordes, esquinas, gradientes)\n- Análisis de nitidez y calidad\n- Dominancia de colores\n- Información técnica de archivo\n\nLIBRERÍAS UTILIZADAS:\n- PIL/Pillow: Carga y metadatos básicos\n- piexif: Metadatos EXIF completos\n- OpenCV: Procesamiento de imagen\n- scikit-image: Análisis avanzado\n- mahotas: Texturas Haralick\n- numpy/scipy: Cál

In [ ]:
import logging
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Union, Tuple
from dataclasses import dataclass, asdict, field
import json
import warnings

import numpy as np
import cv2
from PIL import Image, ImageStat
import piexif
import exifread
import imagehash

from skimage import (
    color, feature, filters, measure,
    morphology, exposure
)
from scipy import ndimage, stats
from sklearn.cluster import KMeans
import mahotas as mh

warnings.filterwarnings('ignore')

In [ ]:
# =============================================================================
# CONFIGURACIÓN DEL SISTEMA
# =============================================================================

@dataclass
class ConfiguracionExtraccion:
    """
    Configuración centralizada del sistema de extracción.

    Attributes:
        ruta_base: Directorio base del proyecto
        ruta_imagenes: Directorio con imágenes a procesar
        ruta_resultados: Directorio para almacenar resultados
        max_dimension_procesamiento: Dimensión máxima para redimensionar (optimización)
        calcular_exif: Activar extracción de metadatos EXIF
        calcular_histogramas: Activar cálculo de histogramas de color
        calcular_texturas: Activar análisis de texturas (Haralick, LBP, GLCM)
        calcular_bordes: Activar detección de bordes y esquinas
        calcular_frecuencias: Activar análisis en dominio de frecuencias
        calcular_ruido: Activar estimación de ruido (SNR, PSNR)
        calcular_saliencia: Activar análisis de saliencia visual
        calcular_colores_dominantes: Activar extracción de paleta de colores
        calcular_hashes: Activar generación de hashes perceptuales
        num_colores_paleta: Número de colores a extraer en paleta
        guardar_json_individual: Guardar JSON por cada imagen
        guardar_json_consolidado: Guardar JSON con todas las imágenes
        nivel_log: Nivel de logging (DEBUG, INFO, WARNING, ERROR)
    """
    ruta_base: Path = Path("/content/drive/MyDrive/TFM/")
    ruta_imagenes: Path = None
    ruta_resultados: Path = None
    max_dimension_procesamiento: int = 2048

    # Flags de procesamiento
    calcular_exif: bool = True
    calcular_histogramas: bool = True
    calcular_texturas: bool = True
    calcular_bordes: bool = True
    calcular_frecuencias: bool = True
    calcular_ruido: bool = True
    calcular_saliencia: bool = True
    calcular_colores_dominantes: bool = True
    calcular_hashes: bool = True

    # Parámetros específicos
    num_colores_paleta: int = 5

    # Opciones de salida
    guardar_json_individual: bool = True
    guardar_json_consolidado: bool = True
    nivel_log: str = "INFO"

    def __post_init__(self):
        """Inicialización post-creación del dataclass"""
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_resultados is None:
            self.ruta_resultados = self.ruta_base / "1_Caracteristicas"

    def crear_estructura_directorios(self) -> None:
        """Crea la estructura de directorios necesaria"""
        self.ruta_resultados.mkdir(parents=True, exist_ok=True)
        (self.ruta_resultados / "json").mkdir(exist_ok=True)
        (self.ruta_resultados / "logs").mkdir(exist_ok=True)

    def validar(self) -> bool:
        """
        Valida la configuración.

        Returns:
            True si la configuración es válida, False en caso contrario
        """
        if not self.ruta_imagenes.exists():
            logging.error(f"Directorio de imágenes no existe: {self.ruta_imagenes}")
            return False

        if self.max_dimension_procesamiento < 256:
            logging.error("max_dimension_procesamiento debe ser >= 256")
            return False

        if self.num_colores_paleta < 1 or self.num_colores_paleta > 20:
            logging.error("num_colores_paleta debe estar entre 1 y 20")
            return False

        return True

In [ ]:
# =============================================================================
# SISTEMA DE LOGGING
# =============================================================================

class LoggerManager:
    """
    Gestor centralizado del sistema de logging.
    """

    def __init__(self, directorio_logs: Path, nivel: str = "INFO"):
        """
        Inicializa el sistema de logging.

        Args:
            directorio_logs: Directorio donde guardar logs
            nivel: Nivel de logging (DEBUG, INFO, WARNING, ERROR)
        """
        self.directorio_logs = directorio_logs
        self.directorio_logs.mkdir(parents=True, exist_ok=True)

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archivo_log = self.directorio_logs / f"extraccion_{timestamp}.log"

        logging.basicConfig(
            level=getattr(logging, nivel.upper()),
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(archivo_log, encoding='utf-8'),
                logging.StreamHandler()
            ]
        )

        self.logger = logging.getLogger('ExtractorCaracteristicas')
        self.logger.info(f"Sistema de logging inicializado. Archivo: {archivo_log.name}")

    def get_logger(self) -> logging.Logger:
        """Retorna el logger configurado"""
        return self.logger


In [ ]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """
    Funciones de utilidad para conversión y serialización de datos.
    """

    @staticmethod
    def convertir_a_serializable(obj: Any) -> Any:
        """
        Convierte objetos a tipos serializables en JSON.

        Maneja tipos numpy, tuplas, listas, bytes y objetos complejos.

        Args:
            obj: Objeto a convertir

        Returns:
            Objeto convertido a tipo serializable
        """
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, (tuple, list)):
            return [Utilidades.convertir_a_serializable(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: Utilidades.convertir_a_serializable(value)
                   for key, value in obj.items()}
        elif isinstance(obj, bytes):
            try:
                return obj.decode('utf-8', errors='ignore')
            except:
                return str(obj)
        elif isinstance(obj, (int, float, str, bool, type(None))):
            return obj
        else:
            return str(obj)

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path) -> bool:
        """
        Guarda diccionario como JSON con manejo de errores.

        Args:
            datos: Diccionario a guardar
            ruta: Ruta del archivo JSON

        Returns:
            True si se guardó correctamente, False en caso contrario
        """
        try:
            datos_serializables = Utilidades.convertir_a_serializable(datos)
            with open(ruta, 'w', encoding='utf-8') as f:
                json.dump(datos_serializables, f, indent=2, ensure_ascii=False)
            return True
        except Exception as e:
            logging.error(f"Error guardando JSON {ruta.name}: {str(e)}")
            return False

    @staticmethod
    def cargar_imagen_robusta(ruta: Path, max_dimension: int = None) -> Tuple[Optional[Image.Image], Optional[np.ndarray], Optional[np.ndarray]]:
        """
        Carga imagen con múltiples fallbacks y redimensionamiento.

        Args:
            ruta: Path a la imagen
            max_dimension: Dimensión máxima permitida (redimensiona si excede)

        Returns:
            Tupla (imagen_pil, imagen_rgb_array, imagen_gray_array)
            Retorna (None, None, None) si falla la carga
        """
        try:
            # Cargar con PIL
            img_pil = Image.open(ruta)

            # Redimensionar si excede max_dimension
            if max_dimension and max(img_pil.size) > max_dimension:
                ratio = max_dimension / max(img_pil.size)
                new_size = tuple(int(dim * ratio) for dim in img_pil.size)
                img_pil = img_pil.resize(new_size, Image.LANCZOS)
                logging.info(f"Imagen {ruta.name} redimensionada de {img_pil.size} a {new_size}")

            # Convertir a RGB si no lo es
            if img_pil.mode != 'RGB':
                img_pil = img_pil.convert('RGB')

            # Convertir a arrays numpy
            img_rgb = np.array(img_pil)
            img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

            return img_pil, img_rgb, img_gray

        except Exception as e:
            logging.error(f"Error cargando imagen {ruta.name}: {str(e)}")
            return None, None, None

In [ ]:
# =============================================================================
# EXTRACTOR DE METADATOS
# =============================================================================

class ExtractorMetadatos:
    """
    Extractor de metadatos de archivo y EXIF.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def extraer_metadatos_archivo(self, ruta: Path, img: Image.Image) -> Dict[str, Any]:
        """
        Extrae metadatos básicos del archivo.

        Args:
            ruta: Path del archivo
            img: Imagen PIL

        Returns:
            Diccionario con metadatos del archivo
        """
        try:
            stats = ruta.stat()
            return {
                'nombre_archivo': str(ruta.name),
                'extension': str(ruta.suffix.lower()),
                'tamaño_bytes': int(stats.st_size),
                'tamaño_mb': round(stats.st_size / (1024 * 1024), 3),
                'fecha_modificacion': datetime.fromtimestamp(stats.st_mtime).isoformat(),
                'formato_imagen': str(img.format) if img.format else 'desconocido',
                'modo_color': str(img.mode),
                'ancho_original': int(img.size[0]),
                'alto_original': int(img.size[1])
            }
        except Exception as e:
            self.logger.error(f"Error extrayendo metadatos de archivo: {str(e)}")
            return {'error': str(e)}

    def extraer_metadatos_exif(self, ruta: Path) -> Dict[str, Any]:
        """
        Extrae metadatos EXIF completos con manejo robusto.

        Args:
            ruta: Path del archivo

        Returns:
            Diccionario con metadatos EXIF
        """
        exif_data = {}

        try:
            # Intentar con piexif
            exif_dict = piexif.load(str(ruta))

            for ifd_name in ["0th", "Exif", "GPS", "1st"]:
                if ifd_name not in exif_dict:
                    continue

                for tag, value in exif_dict[ifd_name].items():
                    try:
                        tag_name = piexif.TAGS[ifd_name][tag]["name"]
                        exif_data[f"{ifd_name}_{tag_name}"] = self._convertir_valor_exif(value)
                    except:
                        continue

        except:
            # Fallback con exifread
            try:
                with open(ruta, 'rb') as f:
                    tags = exifread.process_file(f, details=False)
                    for tag, value in tags.items():
                        if not tag.startswith('Thumbnail'):
                            exif_data[tag] = str(value)
            except Exception as e:
                self.logger.warning(f"No se pudieron extraer metadatos EXIF de {ruta.name}: {str(e)}")

        return exif_data

    def _convertir_valor_exif(self, valor: Any) -> Any:
        """
        Convierte valores EXIF a tipos serializables.

        Args:
            valor: Valor EXIF a convertir

        Returns:
            Valor convertido
        """
        if isinstance(valor, bytes):
            try:
                return valor.decode('utf-8', errors='ignore')
            except:
                return str(valor)
        elif isinstance(valor, (tuple, list)):
            return [self._convertir_valor_exif(v) for v in valor]
        elif isinstance(valor, (int, float, str, bool)):
            return valor
        else:
            return str(valor)

In [ ]:
# =============================================================================
# ANALIZADOR DE COLOR
# =============================================================================

class AnalizadorColor:
    """
    Análisis exhaustivo de características de color.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_estadisticas_color(self, img_pil: Image.Image, img_rgb: np.ndarray) -> Dict[str, Any]:
        """
        Analiza estadísticas de color en múltiples espacios (RGB, HSV, LAB).

        Args:
            img_pil: Imagen PIL
            img_rgb: Imagen como array RGB

        Returns:
            Diccionario con estadísticas de color
        """
        try:
            resultado = {}

            # RGB statistics usando PIL
            stat = ImageStat.Stat(img_pil)
            resultado['rgb'] = {
                'mean': [round(float(x), 2) for x in stat.mean],
                'median': [round(float(x), 2) for x in stat.median],
                'stddev': [round(float(x), 2) for x in stat.stddev],
                'variance': [round(float(x), 2) for x in stat.var]
            }

            # HSV statistics
            img_hsv = color.rgb2hsv(img_rgb / 255.0)
            resultado['hsv'] = {
                'hue_mean': round(float(np.mean(img_hsv[:,:,0])), 4),
                'hue_std': round(float(np.std(img_hsv[:,:,0])), 4),
                'saturation_mean': round(float(np.mean(img_hsv[:,:,1])), 4),
                'saturation_std': round(float(np.std(img_hsv[:,:,1])), 4),
                'value_mean': round(float(np.mean(img_hsv[:,:,2])), 4),
                'value_std': round(float(np.std(img_hsv[:,:,2])), 4)
            }

            # LAB statistics
            img_lab = color.rgb2lab(img_rgb)
            resultado['lab'] = {
                'l_mean': round(float(np.mean(img_lab[:,:,0])), 2),
                'l_std': round(float(np.std(img_lab[:,:,0])), 2),
                'a_mean': round(float(np.mean(img_lab[:,:,1])), 2),
                'a_std': round(float(np.std(img_lab[:,:,1])), 2),
                'b_mean': round(float(np.mean(img_lab[:,:,2])), 2),
                'b_std': round(float(np.std(img_lab[:,:,2])), 2)
            }

            # Métricas globales
            resultado['global'] = {
                'brillo_promedio': round(float(np.mean(img_rgb)), 2),
                'contraste': round(float(np.std(img_rgb)), 2),
                'entropia': round(float(stats.entropy(img_rgb.flatten())), 4)
            }

            return resultado

        except Exception as e:
            self.logger.error(f"Error en análisis de color: {str(e)}")
            return {'error': str(e)}

    def calcular_histogramas(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """
        Calcula histogramas de color por canal.

        Args:
            img_rgb: Imagen RGB como array

        Returns:
            Diccionario con histogramas
        """
        try:
            resultado = {}

            canales = ['rojo', 'verde', 'azul']
            for i, canal in enumerate(canales):
                hist, _ = np.histogram(img_rgb[:,:,i], bins=256, range=(0, 256))
                resultado[canal] = {
                    'histograma': hist.tolist(),
                    'pico_principal': int(np.argmax(hist)),
                    'media': round(float(np.mean(img_rgb[:,:,i])), 2),
                    'std': round(float(np.std(img_rgb[:,:,i])), 2)
                }

            # Histograma de intensidad
            img_gray = color.rgb2gray(img_rgb)
            hist_intensity, _ = np.histogram(img_gray, bins=256, range=(0, 1))
            resultado['intensidad'] = {
                'histograma': hist_intensity.tolist(),
                'entropia': round(float(stats.entropy(hist_intensity + 1)), 4)
            }

            return resultado

        except Exception as e:
            self.logger.error(f"Error calculando histogramas: {str(e)}")
            return {'error': str(e)}

    def extraer_paleta_colores(self, img_pil: Image.Image, n_colores: int = 5) -> Dict[str, Any]:
        """
        Extrae paleta de colores dominantes usando KMeans.

        Args:
            img_pil: Imagen PIL
            n_colores: Número de colores a extraer

        Returns:
            Diccionario con paleta de colores
        """
        try:
            # Redimensionar para acelerar clustering
            img_small = img_pil.copy()
            img_small.thumbnail((150, 150))

            if img_small.mode != 'RGB':
                img_small = img_small.convert('RGB')

            pixels = np.array(img_small).reshape(-1, 3)

            # KMeans clustering
            n_clusters = min(n_colores, len(pixels))
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            kmeans.fit(pixels)

            colors = kmeans.cluster_centers_.astype(int)
            counts = np.bincount(kmeans.labels_)

            # Ordenar por frecuencia
            indices = np.argsort(counts)[::-1]
            colors_ordenados = colors[indices]
            frecuencias = counts[indices] / np.sum(counts)

            return {
                'color_dominante': colors_ordenados[0].tolist(),
                'paleta': colors_ordenados.tolist(),
                'frecuencias': [round(float(f), 4) for f in frecuencias]
            }

        except Exception as e:
            self.logger.error(f"Error extrayendo paleta: {str(e)}")
            return {'error': str(e)}

In [ ]:
# =============================================================================
# ANALIZADOR DE TEXTURAS
# =============================================================================

class AnalizadorTexturas:
    """
    Análisis de texturas usando Haralick, GLCM y LBP.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_texturas(self, img_gray: np.ndarray) -> Dict[str, Any]:
        """
        Análisis completo de texturas.

        Args:
            img_gray: Imagen en escala de grises

        Returns:
            Diccionario con características de textura
        """
        try:
            resultado = {}

            # Normalizar a uint8
            if img_gray.dtype != np.uint8:
                img_gray = (img_gray / img_gray.max() * 255).astype(np.uint8)

            # Haralick features
            resultado['haralick'] = self._calcular_haralick(img_gray)

            # GLCM features
            resultado['glcm'] = self._calcular_glcm(img_gray)

            # LBP features
            resultado['lbp'] = self._calcular_lbp(img_gray)

            # Zernike moments
            resultado['zernike'] = self._calcular_zernike(img_gray)

            return resultado

        except Exception as e:
            self.logger.error(f"Error en análisis de texturas: {str(e)}")
            return {'error': str(e)}

    def _calcular_haralick(self, img: np.ndarray) -> Dict[str, float]:
        """Calcula características de Haralick"""
        try:
            haralick_features = mh.features.haralick(img, return_mean=True)

            nombres = [
                'angular_second_moment', 'contrast', 'correlation',
                'sum_of_squares', 'inverse_diff_moment', 'sum_average',
                'sum_variance', 'sum_entropy', 'entropy', 'difference_variance',
                'difference_entropy', 'info_measure_correlation_1',
                'info_measure_correlation_2'
            ]

            return {nombre: round(float(valor), 6)
                   for nombre, valor in zip(nombres, haralick_features)}
        except:
            return {}

    def _calcular_glcm(self, img: np.ndarray) -> Dict[str, Any]:
        """Calcula características GLCM multi-escala"""
        try:
            distances = [1, 3, 5]
            angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

            glcm = feature.graycomatrix(
                img, distances=distances, angles=angles,
                levels=256, symmetric=True, normed=True
            )

            resultado = {}
            for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']:
                valores = feature.graycoprops(glcm, prop)
                resultado[f'{prop}_mean'] = round(float(np.mean(valores)), 6)
                resultado[f'{prop}_std'] = round(float(np.std(valores)), 6)

            return resultado
        except:
            return {}

    def _calcular_lbp(self, img: np.ndarray) -> Dict[str, float]:
        """Calcula Local Binary Patterns"""
        try:
            lbp = feature.local_binary_pattern(img, P=8, R=1, method='uniform')

            return {
                'mean': round(float(np.mean(lbp)), 4),
                'std': round(float(np.std(lbp)), 4),
                'entropy': round(float(stats.entropy(lbp.flatten())), 6)
            }
        except:
            return {}

    def _calcular_zernike(self, img: np.ndarray) -> List[float]:
        """Calcula momentos de Zernike"""
        try:
            zernike = mh.features.zernike_moments(img, radius=20, degree=8)
            return [round(float(z), 6) for z in zernike]
        except:
            return []

In [ ]:
# =============================================================================
# ANALIZADOR DE BORDES Y CARACTERÍSTICAS VISUALES
# =============================================================================

class AnalizadorBordes:
    """
    Análisis de bordes, esquinas y características geométricas.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_bordes(self, img_gray: np.ndarray) -> Dict[str, Any]:
        """
        Análisis completo de bordes y esquinas.

        Args:
            img_gray: Imagen en escala de grises

        Returns:
            Diccionario con características de bordes
        """
        try:
            resultado = {}

            # Detección de bordes multi-escala
            resultado['canny'] = self._analizar_canny(img_gray)

            # Detección de esquinas
            resultado['esquinas'] = self._detectar_esquinas(img_gray)

            # Análisis de gradientes
            resultado['gradientes'] = self._analizar_gradientes(img_gray)

            # HOG features
            resultado['hog'] = self._calcular_hog(img_gray)

            return resultado

        except Exception as e:
            self.logger.error(f"Error en análisis de bordes: {str(e)}")
            return {'error': str(e)}

    def _analizar_canny(self, img: np.ndarray) -> Dict[str, float]:
        """Detección de bordes Canny multi-escala"""
        try:
            sigmas = [1.0, 2.0, 3.0]
            densidades = []

            for sigma in sigmas:
                edges = feature.canny(img, sigma=sigma)
                densidad = float(np.sum(edges) / edges.size)
                densidades.append(densidad)

            return {
                f'densidad_sigma_{sigma}': round(dens, 6)
                for sigma, dens in zip(sigmas, densidades)
            }
        except:
            return {}

    def _detectar_esquinas(self, img: np.ndarray) -> Dict[str, int]:
        """Detección de esquinas Harris y Shi-Tomasi"""
        try:
            harris = feature.corner_peaks(
                feature.corner_harris(img),
                min_distance=5,
                threshold_rel=0.02
            )

            shi_tomasi = feature.corner_peaks(
                feature.corner_shi_tomasi(img),
                min_distance=5
            )

            return {
                'harris': int(len(harris)),
                'shi_tomasi': int(len(shi_tomasi)),
                'densidad_harris': round(float(len(harris) / img.size), 8)
            }
        except:
            return {}

    def _analizar_gradientes(self, img: np.ndarray) -> Dict[str, float]:
        """Análisis de gradientes con múltiples operadores"""
        try:
            grad_sobel = filters.sobel(img)
            grad_scharr = filters.scharr(img)
            grad_prewitt = filters.prewitt(img)

            return {
                'sobel_mean': round(float(np.mean(grad_sobel)), 6),
                'sobel_max': round(float(np.max(grad_sobel)), 6),
                'sobel_std': round(float(np.std(grad_sobel)), 6),
                'scharr_mean': round(float(np.mean(grad_scharr)), 6),
                'prewitt_mean': round(float(np.mean(grad_prewitt)), 6)
            }
        except:
            return {}

    def _calcular_hog(self, img: np.ndarray) -> Dict[str, float]:
        """Calcula Histogram of Oriented Gradients"""
        try:
            hog_features = feature.hog(
                img, orientations=9,
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2),
                visualize=False
            )

            return {
                'mean': round(float(np.mean(hog_features)), 6),
                'std': round(float(np.std(hog_features)), 6),
                'max': round(float(np.max(hog_features)), 6)
            }
        except:
            return {}


In [ ]:
# =============================================================================
# ANALIZADOR DE CALIDAD
# =============================================================================

class AnalizadorCalidad:
    """
    Análisis de calidad, nitidez y exposición de imagen.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_calidad(self, img_gray: np.ndarray, img_rgb: np.ndarray) -> Dict[str, Any]:
        """
        Análisis completo de calidad de imagen.

        Args:
            img_gray: Imagen en escala de grises
            img_rgb: Imagen RGB

        Returns:
            Diccionario con métricas de calidad
        """
        try:
            resultado = {}

            # Nitidez
            resultado['nitidez'] = self._analizar_nitidez(img_gray)

            # Exposición
            resultado['exposicion'] = self._analizar_exposicion(img_gray)

            # Contraste
            resultado['contraste'] = self._analizar_contraste(img_gray)

            # Balance de blancos
            resultado['balance_blancos'] = self._analizar_balance_blancos(img_rgb)

            # Ruido
            resultado['ruido'] = self._estimar_ruido(img_gray)

            return resultado

        except Exception as e:
            self.logger.error(f"Error en análisis de calidad: {str(e)}")
            return {'error': str(e)}

    def _analizar_nitidez(self, img: np.ndarray) -> Dict[str, float]:
        """Análisis de nitidez con múltiples métricas"""
        try:
            # Laplacian variance
            laplacian = cv2.Laplacian(img.astype(np.float64), cv2.CV_64F)
            nitidez_laplacian = float(np.var(laplacian))

            # Tenengrad (gradiente al cuadrado)
            gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
            gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
            nitidez_tenengrad = float(np.mean(gx**2 + gy**2))

            # Varianza normalizada
            nitidez_varianza = float(np.var(img) / (np.mean(img) + 1e-6))

            return {
                'laplacian': round(nitidez_laplacian, 4),
                'tenengrad': round(nitidez_tenengrad, 4),
                'varianza_normalizada': round(nitidez_varianza, 4)
            }
        except:
            return {}

    def _analizar_exposicion(self, img: np.ndarray) -> Dict[str, float]:
        """Análisis de exposición"""
        try:
            brillo = float(np.mean(img))
            sobre_exp = float(np.sum(img > 250) / img.size * 100)
            sub_exp = float(np.sum(img < 5) / img.size * 100)

            hist, _ = np.histogram(img, bins=256, range=(0, 256))
            hist_norm = hist / np.sum(hist)
            entropia = float(-np.sum(hist_norm * np.log2(hist_norm + 1e-10)))

            return {
                'brillo_medio': round(brillo, 2),
                'sobre_expuesto_pct': round(sobre_exp, 3),
                'sub_expuesto_pct': round(sub_exp, 3),
                'entropia': round(entropia, 4),
                'rango_dinamico': int(img.max() - img.min())
            }
        except:
            return {}

    def _analizar_contraste(self, img: np.ndarray) -> Dict[str, float]:
        """Análisis de contraste"""
        try:
            contraste_rms = float(np.sqrt(np.mean((img - np.mean(img))**2)))
            michelson = float((img.max() - img.min()) / (img.max() + img.min() + 1e-6))

            return {
                'rms': round(contraste_rms, 4),
                'michelson': round(michelson, 6)
            }
        except:
            return {}

    def _analizar_balance_blancos(self, img_rgb: np.ndarray) -> Dict[str, float]:
        """Análisis de balance de blancos"""
        try:
            mean_r = float(np.mean(img_rgb[:,:,0]))
            mean_g = float(np.mean(img_rgb[:,:,1]))
            mean_b = float(np.mean(img_rgb[:,:,2]))

            desviacion = float(np.std([mean_r, mean_g, mean_b]))

            return {
                'mean_red': round(mean_r, 2),
                'mean_green': round(mean_g, 2),
                'mean_blue': round(mean_b, 2),
                'desviacion_canales': round(desviacion, 4)
            }
        except:
            return {}

    def _estimar_ruido(self, img: np.ndarray) -> Dict[str, float]:
        """Estimación de nivel de ruido"""
        try:
            # Laplacian noise estimate
            laplacian = cv2.Laplacian(img, cv2.CV_64F)
            ruido_lap = float(np.std(laplacian))

            # Median filter noise estimate
            median_filtered = ndimage.median_filter(img, size=3)
            noise_estimate = img.astype(float) - median_filtered.astype(float)
            ruido_median = float(np.std(noise_estimate))

            # SNR
            signal = np.mean(img)
            noise = np.std(img)
            snr = float(signal / (noise + 1e-6))
            snr_db = float(20 * np.log10(snr + 1e-6))

            return {
                'laplacian': round(ruido_lap, 4),
                'median_filter': round(ruido_median, 4),
                'snr': round(snr, 4),
                'snr_db': round(snr_db, 2)
            }
        except:
            return {}

In [ ]:
# =============================================================================
# ANALIZADOR DE FRECUENCIAS
# =============================================================================

class AnalizadorFrecuencias:
    """
    Análisis en el dominio de frecuencias usando FFT.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_frecuencias(self, img_gray: np.ndarray) -> Dict[str, Any]:
        """
        Análisis de frecuencias mediante FFT 2D.

        Args:
            img_gray: Imagen en escala de grises

        Returns:
            Diccionario con características espectrales
        """
        try:
            # FFT 2D
            fft = np.fft.fft2(img_gray)
            fft_shift = np.fft.fftshift(fft)
            magnitude = np.abs(fft_shift)

            # Dividir en bandas de frecuencia
            rows, cols = img_gray.shape
            crow, ccol = rows // 2, cols // 2

            y, x = np.ogrid[:rows, :cols]
            distance = np.sqrt((x - ccol)**2 + (y - crow)**2)

            # Máscaras para bandas
            mask_low = distance <= 30
            mask_mid = (distance > 30) & (distance <= 100)
            mask_high = distance > 100

            # Energías por banda
            energia_baja = float(np.sum(magnitude[mask_low]))
            energia_media = float(np.sum(magnitude[mask_mid]))
            energia_alta = float(np.sum(magnitude[mask_high]))
            energia_total = energia_baja + energia_media + energia_alta

            # Ratios
            ratio_baja = float(energia_baja / energia_total) if energia_total > 0 else 0
            ratio_media = float(energia_media / energia_total) if energia_total > 0 else 0
            ratio_alta = float(energia_alta / energia_total) if energia_total > 0 else 0

            # Entropía espectral
            spectrum_norm = magnitude / (np.sum(magnitude) + 1e-10)
            entropia = float(-np.sum(spectrum_norm * np.log2(spectrum_norm + 1e-10)))

            return {
                'energia_frecuencia_baja': round(energia_baja, 2),
                'energia_frecuencia_media': round(energia_media, 2),
                'energia_frecuencia_alta': round(energia_alta, 2),
                'ratio_baja': round(ratio_baja, 6),
                'ratio_media': round(ratio_media, 6),
                'ratio_alta': round(ratio_alta, 6),
                'entropia_espectral': round(entropia, 6),
                'pico_dominante': round(float(np.max(magnitude)), 2)
            }

        except Exception as e:
            self.logger.error(f"Error en análisis de frecuencias: {str(e)}")
            return {'error': str(e)}

In [ ]:
# =============================================================================
# EXTRACTOR PRINCIPAL
# =============================================================================

class ExtractorCaracteristicasImagen:
    """
    Clase principal que orquesta la extracción de características.
    """

    def __init__(self, config: ConfiguracionExtraccion):
        """
        Inicializa el extractor.

        Args:
            config: Configuración del sistema
        """
        self.config = config
        self.config.crear_estructura_directorios()

        # Inicializar logger
        self.logger_manager = LoggerManager(
            self.config.ruta_resultados / "logs",
            self.config.nivel_log
        )
        self.logger = self.logger_manager.get_logger()

        # Inicializar analizadores
        self.extractor_metadatos = ExtractorMetadatos(self.logger)
        self.analizador_color = AnalizadorColor(self.logger)
        self.analizador_texturas = AnalizadorTexturas(self.logger)
        self.analizador_bordes = AnalizadorBordes(self.logger)
        self.analizador_calidad = AnalizadorCalidad(self.logger)
        self.analizador_frecuencias = AnalizadorFrecuencias(self.logger)
        self.analizador_saliencia = AnalizadorSaliencia(self.logger)

        self.resultados = []

        self.logger.info("Sistema de extracción inicializado correctamente")

    def procesar_imagen(self, ruta_imagen: Path) -> Dict[str, Any]:
        """
        Procesa una imagen completa.

        Args:
            ruta_imagen: Path a la imagen

        Returns:
            Diccionario con todas las características extraídas
        """
        self.logger.info(f"Procesando imagen: {ruta_imagen.name}")
        inicio = datetime.now()

        try:
            # Cargar imagen
            img_pil, img_rgb, img_gray = Utilidades.cargar_imagen_robusta(
                ruta_imagen,
                self.config.max_dimension_procesamiento
            )

            if img_pil is None:
                raise ValueError("Error cargando imagen")

            # Estructura de resultados
            caracteristicas = {
                'metadatos_archivo': self.extractor_metadatos.extraer_metadatos_archivo(ruta_imagen, img_pil)
            }

            # Metadatos EXIF
            if self.config.calcular_exif:
                caracteristicas['metadatos_exif'] = self.extractor_metadatos.extraer_metadatos_exif(ruta_imagen)

            # Estadísticas de color
            caracteristicas['estadisticas_color'] = self.analizador_color.analizar_estadisticas_color(img_pil, img_rgb)

            # Histogramas
            if self.config.calcular_histogramas:
                caracteristicas['histogramas'] = self.analizador_color.calcular_histogramas(img_rgb)

            # Paleta de colores
            if self.config.calcular_colores_dominantes:
                caracteristicas['colores_dominantes'] = self.analizador_color.extraer_paleta_colores(
                    img_pil,
                    self.config.num_colores_paleta
                )

            # Texturas
            if self.config.calcular_texturas:
                caracteristicas['texturas'] = self.analizador_texturas.analizar_texturas(img_gray)

            # Bordes
            if self.config.calcular_bordes:
                caracteristicas['bordes_caracteristicas'] = self.analizador_bordes.analizar_bordes(img_gray)

            # Calidad
            caracteristicas['calidad'] = self.analizador_calidad.analizar_calidad(img_gray, img_rgb)

            # Frecuencias
            if self.config.calcular_frecuencias:
                caracteristicas['analisis_frecuencia'] = self.analizador_frecuencias.analizar_frecuencias(img_gray)

            # Saliencia visual
            if self.config.calcular_saliencia:
                caracteristicas['saliencia_visual'] = self.analizador_saliencia.analizar_saliencia(img_rgb)

            # Hashes perceptuales
            if self.config.calcular_hashes:
                caracteristicas['hashes_perceptuales'] = self._calcular_hashes(img_pil)

            # Timestamp y tiempo de procesamiento
            tiempo_procesamiento = (datetime.now() - inicio).total_seconds()
            caracteristicas['procesamiento'] = {
                'timestamp': datetime.now().isoformat(),
                'tiempo_segundos': round(tiempo_procesamiento, 3)
            }

            self.logger.info(f"Imagen procesada exitosamente en {tiempo_procesamiento:.2f}s")
            return caracteristicas

        except Exception as e:
            self.logger.error(f"Error procesando {ruta_imagen.name}: {str(e)}")
            return {
                'archivo': str(ruta_imagen.name),
                'error': str(e),
                'timestamp': datetime.now().isoformat()
            }

    def _calcular_hashes(self, img: Image.Image) -> Dict[str, str]:
        """Calcula hashes perceptuales para similitud"""
        try:
            return {
                'average_hash': str(imagehash.average_hash(img)),
                'perceptual_hash': str(imagehash.phash(img)),
                'difference_hash': str(imagehash.dhash(img)),
                'wavelet_hash': str(imagehash.whash(img)),
                'color_hash': str(imagehash.colorhash(img))
            }
        except:
            return {}

    def procesar_directorio(self) -> List[Dict[str, Any]]:
        """
        Procesa todas las imágenes del directorio configurado.

        Returns:
            Lista de diccionarios con características de cada imagen
        """
        extensiones = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
        imagenes = []

        for ext in extensiones:
            imagenes.extend(self.config.ruta_imagenes.glob(ext))

        if not imagenes:
            self.logger.warning(f"No se encontraron imágenes en {self.config.ruta_imagenes}")
            return []

        self.logger.info(f"Iniciando procesamiento de {len(imagenes)} imágenes")

        resultados = []
        for i, img_path in enumerate(imagenes, 1):
            self.logger.info(f"[{i}/{len(imagenes)}] Procesando {img_path.name}")

            resultado = self.procesar_imagen(img_path)
            resultados.append(resultado)

            # Guardar JSON individual si está configurado
            if self.config.guardar_json_individual:
                json_path = self.config.ruta_resultados / "json" / f"{img_path.stem}_caracteristicas.json"
                Utilidades.guardar_json(resultado, json_path)

        self.resultados = resultados
        self.logger.info(f"Procesamiento completado: {len(resultados)} imágenes")

        return resultados

    def guardar_consolidado(self, nombre_archivo: str = None) -> Optional[Path]:
        """
        Guarda todos los resultados en un JSON consolidado.

        Args:
            nombre_archivo: Nombre del archivo (opcional)

        Returns:
            Path del archivo guardado o None si falla
        """
        if not self.resultados:
            self.logger.warning("No hay resultados para guardar")
            return None

        if nombre_archivo is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            nombre_archivo = f"caracteristicas_consolidadas_{timestamp}.json"

        ruta_salida = self.config.ruta_resultados / "json" / nombre_archivo

        datos_consolidados = {
            'metadata': {
                'timestamp_generacion': datetime.now().isoformat(),
                'total_imagenes': len(self.resultados),
                'imagenes_exitosas': len([r for r in self.resultados if 'error' not in r]),
                'imagenes_con_error': len([r for r in self.resultados if 'error' in r]),
                'ruta_origen': str(self.config.ruta_imagenes),
                'configuracion': Utilidades.convertir_a_serializable(asdict(self.config))
            },
            'imagenes': self.resultados
        }

        if Utilidades.guardar_json(datos_consolidados, ruta_salida):
            tamaño_kb = ruta_salida.stat().st_size / 1024
            self.logger.info(f"Archivo consolidado guardado: {ruta_salida.name} ({tamaño_kb:.2f} KB)")
            return ruta_salida

        return None

    def generar_reporte(self) -> Dict[str, Any]:
        """
        Genera reporte estadístico del procesamiento.

        Returns:
            Diccionario con estadísticas del procesamiento
        """
        if not self.resultados:
            return {}

        exitosas = [r for r in self.resultados if 'error' not in r]
        con_error = [r for r in self.resultados if 'error' in r]

        tiempos = [r.get('procesamiento', {}).get('tiempo_segundos', 0)
                  for r in exitosas]

        return {
            'total_imagenes': len(self.resultados),
            'exitosas': len(exitosas),
            'con_error': len(con_error),
            'tiempo_total_segundos': round(sum(tiempos), 2),
            'tiempo_promedio_segundos': round(np.mean(tiempos), 2) if tiempos else 0,
            'tiempo_min_segundos': round(min(tiempos), 2) if tiempos else 0,
            'tiempo_max_segundos': round(max(tiempos), 2) if tiempos else 0
        }

In [ ]:
# =============================================================================
# ANALIZADOR DE SALIENCIA VISUAL
# =============================================================================

class AnalizadorSaliencia:
    """
    Análisis de saliencia visual - regiones que atraen la atención.
    Implementa Spectral Residual Saliency y análisis de distribución espacial.
    """

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def analizar_saliencia(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """
        Análisis completo de saliencia visual.

        La saliencia detecta regiones de la imagen que naturalmente atraen
        la atención visual humana, útil para entender composición y puntos focales.

        Args:
            img_rgb: Imagen RGB como array

        Returns:
            Diccionario con métricas de saliencia
        """
        try:
            resultado = {}

            # Saliencia espectral (Spectral Residual)
            resultado['espectral'] = self._calcular_saliencia_espectral(img_rgb)

            # Análisis de distribución espacial de saliencia
            resultado['distribucion_espacial'] = self._analizar_distribucion_espacial(img_rgb)

            # Centroide de atención visual
            resultado['centroide'] = self._calcular_centroide_atencion(img_rgb)

            return resultado

        except Exception as e:
            self.logger.error(f"Error en análisis de saliencia: {str(e)}")
            return {'error': str(e)}

    def _calcular_saliencia_espectral(self, img_rgb: np.ndarray) -> Dict[str, float]:
        """
        Calcula saliencia usando Spectral Residual method.

        Basado en: Hou, X., & Zhang, L. (2007). Saliency detection:
        A spectral residual approach. CVPR 2007.
        """
        try:
            # Convertir a escala de grises
            img_gray = color.rgb2gray(img_rgb)

            # FFT
            fft = np.fft.fft2(img_gray)
            magnitude = np.abs(fft)
            phase = np.angle(fft)

            # Log spectrum
            log_amplitude = np.log(magnitude + 1e-10)

            # Spectral residual (diferencia con promedio suavizado)
            avg_filter_size = 3
            avg_log_amplitude = ndimage.uniform_filter(log_amplitude, size=avg_filter_size)
            spectral_residual = log_amplitude - avg_log_amplitude

            # Reconstruir con residuo
            saliency_fft = np.exp(spectral_residual + 1j * phase)
            saliency_map = np.abs(np.fft.ifft2(saliency_fft))**2

            # Normalizar
            saliency_map = (saliency_map - saliency_map.min()) / (saliency_map.max() - saliency_map.min() + 1e-10)

            # Suavizar resultado
            saliency_map = ndimage.gaussian_filter(saliency_map, sigma=2)

            # Métricas
            return {
                'media': round(float(np.mean(saliency_map)), 6),
                'std': round(float(np.std(saliency_map)), 6),
                'max': round(float(np.max(saliency_map)), 6),
                'percentil_90': round(float(np.percentile(saliency_map, 90)), 6),
                'percentil_95': round(float(np.percentile(saliency_map, 95)), 6)
            }
        except:
            return {}

    def _analizar_distribucion_espacial(self, img_rgb: np.ndarray) -> Dict[str, Any]:
        """
        Analiza cómo se distribuye la saliencia en el espacio de la imagen.
        """
        try:
            img_gray = color.rgb2gray(img_rgb)

            # Calcular mapa de saliencia simple basado en contraste local
            # Usando varianza local como proxy de saliencia
            window_size = 15
            saliency_local = ndimage.generic_filter(
                img_gray,
                np.var,
                size=window_size
            )

            # Normalizar
            saliency_local = (saliency_local - saliency_local.min()) / (saliency_local.max() - saliency_local.min() + 1e-10)

            # Dividir imagen en cuadrantes
            h, w = img_gray.shape
            h2, w2 = h // 2, w // 2

            cuadrantes = {
                'superior_izquierda': saliency_local[:h2, :w2],
                'superior_derecha': saliency_local[:h2, w2:],
                'inferior_izquierda': saliency_local[h2:, :w2],
                'inferior_derecha': saliency_local[h2:, w2:]
            }

            # Saliencia promedio por cuadrante
            distribucion = {}
            for nombre, cuadrante in cuadrantes.items():
                distribucion[f'saliencia_{nombre}'] = round(float(np.mean(cuadrante)), 6)

            # Calcular variabilidad entre cuadrantes
            valores_cuadrantes = [np.mean(c) for c in cuadrantes.values()]
            distribucion['variabilidad_cuadrantes'] = round(float(np.std(valores_cuadrantes)), 6)

            # Índice de centralidad (si saliencia está concentrada en el centro)
            centro_h_start, centro_h_end = h // 4, 3 * h // 4
            centro_w_start, centro_w_end = w // 4, 3 * w // 4
            saliencia_centro = saliency_local[centro_h_start:centro_h_end, centro_w_start:centro_w_end]
            saliencia_periferia = np.concatenate([
                saliency_local[:centro_h_start, :].flatten(),
                saliency_local[centro_h_end:, :].flatten(),
                saliency_local[:, :centro_w_start].flatten(),
                saliency_local[:, centro_w_end:].flatten()
            ])

            distribucion['saliencia_centro'] = round(float(np.mean(saliencia_centro)), 6)
            distribucion['saliencia_periferia'] = round(float(np.mean(saliencia_periferia)), 6)
            distribucion['ratio_centro_periferia'] = round(
                float(np.mean(saliencia_centro) / (np.mean(saliencia_periferia) + 1e-10)),
                6
            )

            return distribucion
        except:
            return {}

    def _calcular_centroide_atencion(self, img_rgb: np.ndarray) -> Dict[str, float]:
        """
        Calcula el centroide de atención visual (dónde se concentra la saliencia).

        Retorna coordenadas normalizadas [0, 1] del punto focal de atención.
        """
        try:
            img_gray = color.rgb2gray(img_rgb)
            h, w = img_gray.shape

            # Mapa de saliencia usando contraste local
            saliency = ndimage.generic_filter(img_gray, np.var, size=15)
            saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-10)

            # Umbral para región saliente (percentil 90)
            threshold = np.percentile(saliency, 90)
            salient_region = saliency > threshold

            # Calcular centroide de región saliente
            if np.any(salient_region):
                y_coords, x_coords = np.where(salient_region)

                # Centroide ponderado por intensidad de saliencia
                weights = saliency[salient_region]
                centroide_y = float(np.average(y_coords, weights=weights) / h)
                centroide_x = float(np.average(x_coords, weights=weights) / w)

                # Área de región saliente
                area_saliente = float(np.sum(salient_region) / salient_region.size * 100)
            else:
                # Si no hay región saliente clara, usar centro de imagen
                centroide_y = 0.5
                centroide_x = 0.5
                area_saliente = 0.0

            # Distancia del centroide al centro geométrico de la imagen
            distancia_centro = float(np.sqrt((centroide_x - 0.5)**2 + (centroide_y - 0.5)**2))

            return {
                'centroide_x_normalizado': round(centroide_x, 4),
                'centroide_y_normalizado': round(centroide_y, 4),
                'distancia_desde_centro': round(distancia_centro, 4),
                'area_saliente_porcentaje': round(area_saliente, 2)
            }
        except:
            return {}

In [ ]:
# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def main():
    """
    Función principal de ejecución del sistema.
    """
    # Montar Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
    except:
        pass

    # Configurar sistema
    config = ConfiguracionExtraccion(
        ruta_base=Path("/content/drive/MyDrive/TFM/"),
        max_dimension_procesamiento=2048,
        calcular_exif=True,
        calcular_histogramas=True,
        calcular_texturas=True,
        calcular_bordes=True,
        calcular_frecuencias=True,
        calcular_ruido=True,
        calcular_saliencia=True,
        calcular_colores_dominantes=True,
        calcular_hashes=True,
        num_colores_paleta=5,
        guardar_json_individual=True,
        guardar_json_consolidado=True,
        nivel_log="INFO"
    )

    # Validar configuración
    if not config.validar():
        print("ERROR: Configuración inválida")
        return None

    # Crear extractor
    extractor = ExtractorCaracteristicasImagen(config)

    # Procesar imágenes
    resultados = extractor.procesar_directorio()

    # Guardar consolidado
    if resultados and config.guardar_json_consolidado:
        extractor.guardar_consolidado()

    # Generar reporte
    reporte = extractor.generar_reporte()

    print("\n" + "="*80)
    print("RESUMEN DE PROCESAMIENTO")
    print("="*80)
    print(f"Total imágenes: {reporte.get('total_imagenes', 0)}")
    print(f"Exitosas: {reporte.get('exitosas', 0)}")
    print(f"Con errores: {reporte.get('con_error', 0)}")
    print(f"Tiempo total: {reporte.get('tiempo_total_segundos', 0):.2f}s")
    print(f"Tiempo promedio: {reporte.get('tiempo_promedio_segundos', 0):.2f}s")
    print(f"Resultados: {config.ruta_resultados / 'json'}")
    print("="*80)

    return resultados

In [ ]:
if __name__ == "__main__":
    resultados = main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

RESUMEN DE PROCESAMIENTO
Total imágenes: 23
Exitosas: 23
Con errores: 0
Tiempo total: 3450.17s
Tiempo promedio: 150.01s
Resultados: /content/drive/MyDrive/TFM/1_Caracteristicas/json
